In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d jaiahuja/skin-cancer-detection

100% 784M/786M [00:42<00:00, 20.9MB/s]
100% 786M/786M [00:42<00:00, 19.3MB/s]


In [3]:
from zipfile import ZipFile
dataset = '/content/skin-cancer-detection.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [10]:
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
from tqdm import tqdm

In [5]:
train_folder=os.listdir('/content/Skin cancer ISIC The International Skin Imaging Collaboration/Train')
print(train_folder)
print(len(train_folder))

['actinic keratosis', 'vascular lesion', 'squamous cell carcinoma', 'seborrheic keratosis', 'nevus', 'dermatofibroma', 'pigmented benign keratosis', 'melanoma', 'basal cell carcinoma']
9


In [9]:
data=[]

In [11]:
for i in tqdm(range(9)):
  folder_path='/content/Skin cancer ISIC The International Skin Imaging Collaboration/Train/'+train_folder[i]
  img_folder=os.listdir(folder_path)
  l=len(img_folder)
  for j in range(l):
    img_path=folder_path+'/'+img_folder[j]
    img_pillow=Image.open(img_path)
    img_color=img_pillow.convert('RGB')
    img_resize=img_color.resize((64,64))
    img_arr=np.array(img_resize)
    data.append([img_arr,i])

100%|██████████| 9/9 [00:37<00:00,  4.16s/it]


In [12]:
x=[]

In [13]:
y=[]

In [14]:
for img_array,label in data:
  x.append(img_array)
  y.append(label)

In [15]:
x=np.array(x)
y=np.array(y)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=0)

In [17]:
x_train_scaled=x_train/255
x_test_scaled=x_test/255

In [18]:
import tensorflow as tf
from tensorflow import keras

In [19]:
num_of_classes=10
model=keras.Sequential()

model.add(keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(64,64,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_of_classes,activation='softmax'))

In [20]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(x_train_scaled,y_train,epochs=100,validation_split=0.1)

Epoch 1/100
57/57 [==============================] - 1s 9ms/step - loss: 1.5139 - accuracy: 0.4617 - val_loss: 1.4205 - val_accuracy: 0.5198
Epoch 2/100
57/57 [==============================] - 0s 7ms/step - loss: 1.6111 - accuracy: 0.4418 - val_loss: 1.4297 - val_accuracy: 0.5495
Epoch 3/100
57/57 [==============================] - 0s 7ms/step - loss: 1.5427 - accuracy: 0.4727 - val_loss: 1.4381 - val_accuracy: 0.5297
Epoch 4/100
57/57 [==============================] - 0s 6ms/step - loss: 1.4877 - accuracy: 0.4837 - val_loss: 1.3607 - val_accuracy: 0.5446
Epoch 5/100
57/57 [==============================] - 0s 7ms/step - loss: 1.4552 - accuracy: 0.4986 - val_loss: 1.4140 - val_accuracy: 0.5198
Epoch 6/100
57/57 [==============================] - 0s 6ms/step - loss: 1.4208 - accuracy: 0.5080 - val_loss: 1.4086 - val_accuracy: 0.5297
Epoch 7/100
57/57 [==============================] - 0s 6ms/step - loss: 1.4225 - accuracy: 0.5124 - val_loss: 1.3285 - val_accuracy: 0.5495
Epoch 8/100
5

In [25]:
img_path='/content/Skin cancer ISIC The International Skin Imaging Collaboration/Train/actinic keratosis/ISIC_0025780.jpg'
img_pillow=Image.open(img_path)
img_color=img_pillow.convert('RGB')
img_resize=img_color.resize((64,64))
img_arr=np.array(img_resize)
img_scaled=img_arr/255
img_reshape=np.reshape(img_scaled,[1,64,64,3])
prediction=model.predict(img_reshape)
print(prediction)
output=np.argmax(prediction)
print(output)
if(output==0):
  print('The skin cancer type is actinic keratosis')
elif(output==1):
  print('The skin cancer type is vascular lesion')
elif(output==2):
  print('The skin cancer type is squamous cell carcinoma')
elif(output==3):
  print('The skin cancer type is seborrheic keratosis')
elif(output==4):
  print('The skin cancer type is nevus')
elif(output==5):
  print('The skin cancer type is dermatofibroma')
elif(output==6):
  print('The skin cancer type is pigmented benign keratosis')
elif(output==7):
  print('The skin cancer type is melanoma')
elif(output==8):
  print('The skin cancer type is basal cell carcinoma')



1/1 [==============================] - 0s 18ms/step
[[5.8055741e-01 8.6218570e-05 5.7502404e-02 1.2357114e-03 3.3059579e-01
  1.0375955e-02 1.3581662e-02 5.4238769e-03 6.3910021e-04 1.8342773e-06]]
0
The skin cancer type is actinic keratosis
